# Multi-Commodity Flow

**Выполнила: Ковалева Варвара**

## Постановка задачи

### Дано:
- Ориентированный граф $ G = (V, E) $
- Множество товаров (commodities) $ K $
- Для каждого товара $ k \in K $: источник $ s_k $, сток $ t_k $, требуемый поток $ d_k $
- Для каждого ребра $ e \in E $: стоимость использования одним ТС $ c_e $, вместимость ТС $ C $
- Для каждого узла $ v \in V $: $ f_v \in \mathbb{R}_+ $ — стоимость перегруза единицы груза в узле $ v $, $ W_v \in \mathbb{R}_+ $ — максимальное количество перегруза в узле $ v $
- Для каждого товара $ k \in K $: $ \Pi_k $ — множество всех возможных путей из $ s_k $ в $ t_k $ в графе $ G $

---

### Переменные:
- $ x^k_p \in \mathbb{R}_+ $ — поток товара $ k $ по пути $ p \in \Pi_k $
- $ y_e \in \mathbb{Z}_+ $ — количество ТС, используемых на ребре $ e \in E $

---

### Целевая функция:
$$
\min \left(
\sum_{e \in E} c_e \cdot y_e +
\sum_{k \in K} \sum_{p \in \Pi_k}  \sum_{\substack{v \in p \\ v \neq s_k \\ v \neq t_k}}  f_v \cdot x^k_p
\right)
$$

---

### Ограничения:

1. **Полный поток для каждого товара:**
   $$
   \sum_{p \in \Pi_k} x^k_p = d_k \quad \forall k \in K
   $$
   *Обеспечивает удовлетворение спроса для каждого товара.*

2. **Связь между потоком и количеством ТС:**
   $$
   \sum_{k \in K} \sum_{p \in \Pi_k : e \in p} x^k_p \leq C \cdot y_e \quad \forall e \in E, \quad y_e \in \mathbb{Z}_+
   $$
   *Количество ТС $ y_e $ определяется суммарным потоком на ребре $ e $.*

3. **Неотрицательность потоков:**
   $$
   x^k_p \geq 0 \quad \forall k \in K, \, \forall p \in \Pi_k
   $$

4. **Максимальная перегрузка в узле**
   $$
   \sum_{k \in K} \sum_{p \in \Pi_k}  \sum_{\substack{v \in p \\ v \neq s_k \\ v \neq t_k}} x^k_p \le W_e \quad \forall e \in E
   $$

In [33]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import time

import pulp
from highspy import Highs
from highspy._core import HighsModelStatus  # Импортируем константы статуса

from ortools.linear_solver import pywraplp

import gurobipy as gp
from gurobipy import GRB


## Подход к решению

In [ ]:
def solve_min_cost_flow(data, solver_data): # Реализация солвера без учета ограничения на максимальный вес перегрузки
  # Создаем модель
  model = pulp.LpProblem('MinCostFlow', pulp.LpMinimize)

  # Переменные
  x = {}
  for pid in solver_data.product_path_ids:
        lb = solver_data.flow_lb[pid]
        ub = solver_data.flow_ub[pid]
        x[pid] = pulp.LpVariable(f"flow_{pid}", lowBound=lb, upBound=ub, cat='Continuous')

  y = pulp.LpVariable.dicts('vehicles_count', data.edges, lowBound=0, cat='Integer')

  # Целевая функция
  model += (
      pulp.lpSum(data.edges_price[e] * y[e] for e in data.edges) +
      pulp.lpSum(
          pulp.lpSum(data.offices[v][0] * x[x_i_p] for v in solver_data.paths[x_i_p][1:-1])
          for x_i_p in solver_data.product_path_ids
      ),
      "Total_Cost"
  )

  # Ограничения
  # 1. Поток для каждого продукта должен удовлетворять спрос
  for product in data.products_ids:
      model += (
          pulp.lpSum(x[(product, p_i)] for p_i in range(solver_data.product_path_count[product])) == data.products[product][2],
          f"Demand_{product}"
      )

  # 2. Подготовка edges_paths и offices_paths
  edges_paths = {edge: set() for edge in data.edges}
  offices_paths = {office: set() for office in data.offices_ids}
  for product_path_id, path in solver_data.paths.items():
      for i in range(len(path) - 1):
          edges_paths[(path[i], path[i + 1])].add(product_path_id)
          if i != 0:
              offices_paths[path[i]].add(product_path_id)

  # 3. Ограничение на вместимость транспорта
  for e in data.edges:
      model += (
          pulp.lpSum(x[x_i_p] for x_i_p in edges_paths[e]) <= data.vehicle_capacity * y[e],
          f"Capacity_{e}"
      )

  # 4. Ограничение на пропускную способность узлов (если есть limited_offices)
  for office in solver_data.limited_offices:
      model += (
          pulp.lpSum(x[x_i_p] for x_i_p in offices_paths[office]) <= data.offices[office][1],
          f"Office_Capacity_{office}"
      )


  # Используем HiGHS как солвер
  solver = pulp.HiGHS(
    msg=True,
    mip=True,              
    timeLimit=7200.0,
    mip_rel_gap = 0.01,
    parallel = 1)          
  
  print(f"Переменных: {len(model.variables())}")
  print(f"Ограничений: {len(model.constraints)}")

  model.solve(solver)

  # Проверка статуса решения
  if pulp.LpStatus[model.status] == 'Optimal':
      flow = {product_path_id: x[product_path_id].varValue for product_path_id in solver_data.product_path_ids if x[product_path_id].varValue > 0}
      v_count = {e: y[e].varValue for e in data.edges}
      obj_val = pulp.value(model.objective)
      return flow, v_count, obj_val
  else:
      return None, None, None

В кратце алгоритм можно разбить на два этапа:
1. Поиск результата без учета огрничения по максимальной перегрузке в узле
2. Разгрузка перегруженных узлов с помощью итеративного исключения их из графа и сохранения новых сгенерированных путей
3. Решение солвера со всеми накопленными путями и всеми ограничениями

### 1. Расчёт результата без учета максимальной перегрузки в узлах

После считывания данных и создания графа в виде списка ребер с стоимостями, модифицируем немного это список следующим образом: для каждого ребра добавим стоимости перегруза его концов. Это нужно для того, чтобы учесть стоимость перегруза в поиске крачайших путей.

Собственно, следующий шаг и есть поиск k кратчайших путей.

Затем с помощью солвера находим решение в путевой постановке задачи с использование найденных k кратчайших путей.

In [2]:
class Data: # Общие данные задачи
  def __init__(self, vehicle_capacity, folder):
    self.offices_ids = []
    self.products_ids = []
    self.edges = []
    self.offices = {}
    self.products = {}
    self.edges_price = {}
    self.vehicle_capacity = vehicle_capacity

    df_offices = pd.read_csv(folder+'offices.csv', usecols=['office_id','transfer_price','transfer_max'], dtype={'office_id':int,'transfer_price':float, 'transfer_max':int})
    df_reqs = pd.read_csv(folder+'reqs.csv', dtype={'src_office_id':int,'dst_office_id':int,'volume':int})
    df_distance_matrix = pd.read_csv(folder+'distance_matrix.csv', usecols =['src','dst','price'], dtype={'src':int,'dst':int, 'price':float})

    for(i, row) in df_offices.iterrows():
      self.offices_ids.append(row['office_id'])
      self.offices[row['office_id']] = (row['transfer_price'], row['transfer_max'])

    for(i, row) in df_reqs.iterrows():
      self.products_ids.append(i)
      self.products[i] = (row['src_office_id'], row['dst_office_id'], row['volume'])

    for(i, row) in df_distance_matrix.iterrows():
      if(row['src'] != row['dst'] and row['price']>0):
        edge = (row['src'], row['dst'])
        self.edges.append(edge)
        self.edges_price[edge] = row['price']


In [3]:
class SolverData: # Данные, необходимые для солвера
  def __init__(self):
    self.product_path_ids = [] # массив (индекс товара - индекс пути)
    self.product_path_count = {} # товар - количество путей
    self.paths = {} # рассматриваемые пути продуктов
    self.limited_offices = set()
    self.flow_lb = {}
    self.flow_ub = {}

In [4]:
def count_price_of_edjes(edges_price, offices): # Расчет стоимостей ребер с учетом стоимостей перегруза в вершинах
  edges_with_transfer_price = edges_price.copy()
  for edge in edges_price:
    edges_with_transfer_price[edge] += offices[edge[0]][0]+offices[edge[1]][0]
  return edges_with_transfer_price

In [5]:
def edges_to_nx_graph(edges_price):
    G = nx.DiGraph()
    for edge, price in edges_price.items():
      G.add_edge(edge[0], edge[1], weight = price)
    return G

Хоть мы и прибавили к каждому ребру стоимость перегрузки в его вершинах, стоит не забывать, что мы не учитываем эту стоимость в source и target вершинах. Поэтому в следующем методе как раз произходит вычитание их стоимостей из смежных с ними ребер.

In [6]:
def remove_transfer_price_source_target(edges_with_transfer_price, source, target, offices):
  edges_for_product = edges_with_transfer_price.copy()

  price_source = offices[source][0]
  price_target = offices[target][0]

  prices = [price_source, price_source, price_target, price_target]

  for v in offices:
    edges = [(source, v), (v, source), (target, v), (v, target)]
    for i in range(4):
      if(edges[i] in edges_for_product):
        edges_for_product[edges[i]] = edges_for_product[edges[i]] - prices[i]

  return edges_for_product

In [7]:
def calc_product_paths(edges_with_transfer_price, source, target, k, office): # Расчёт k-кратчайших путей для конкретного товара
  edges_for_product = remove_transfer_price_source_target(edges_with_transfer_price, source, target, office)

  graph = edges_to_nx_graph(edges_for_product)
  paths_gen = nx.shortest_simple_paths(graph, source=source, target=target, weight='weight')
  paths_list = list(itertools.islice(paths_gen, k))

  return paths_list


In [8]:
def calc_paths(edges_with_transfer_price, data, count_of_paths): # Расчёт кратчайших путей для всех товаров
  paths = {}

  for product, product_data in data.products.items():
    paths[product] = calc_product_paths(edges_with_transfer_price, product_data[0], product_data[1], count_of_paths, data.offices)

  return paths

In [9]:
def make_solver_data(paths, products): # Формирование данных для солвера
  solver_data = SolverData()

  for product, product_paths in paths.items():
    solver_data.product_path_count[product] = len(product_paths)
    for i, path in enumerate(product_paths):
      solver_data.product_path_ids.append((product, i))
      solver_data.paths[(product, i)] = path

  max_ub = -1
  for product, product_data in products.items():
    max_ub = max(max_ub, product_data[2])

  solver_data.flow_lb = {product_path_id : 0 for product_path_id in solver_data.product_path_ids }
  solver_data.flow_ub = {product_path_id : max_ub for product_path_id in solver_data.product_path_ids }

  return solver_data

In [10]:
import gurobipy as gp
from gurobipy import GRB

In [11]:
def solve_min_cost_flow_gurobi(data, solver_data): # Реализация солвера без учета ограничения на максимальный вес перегрузки

  model = gp.Model('MinCostFlow')

  # Переменные
  x = model.addVars(solver_data.product_path_ids, name='flow', lb = solver_data.flow_lb, ub = solver_data.flow_ub, vtype = GRB.CONTINUOUS)
  y = model.addVars(data.edges, name='vehicles count', lb=0, vtype = GRB.INTEGER)

  # Целевая функция
  model.setObjective(
      gp.quicksum(data.edges_price[e]*y[e] for e in data.edges) + gp.quicksum(gp.quicksum(data.offices[v][0]*x[x_i_p] for v in solver_data.paths[x_i_p][1:-1]) for x_i_p in solver_data.product_path_ids),
        GRB.MINIMIZE
    )

  # Ограничения

  for product in data.products_ids:
    model.addConstr(gp.quicksum(x[(product,p_i)] for p_i in range(solver_data.product_path_count[product])) == data.products[product][2])


  edges_paths = {edge: set() for edge in data.edges}
  offices_paths = {office: set() for office in data.offices_ids}
  for product_path_id, path in solver_data.paths.items():
    for i in range(len(path)-1):
      edges_paths[(path[i], path[i+1])].add(product_path_id)
      if(i!=0):
        offices_paths[path[i]].add(product_path_id)



  for e in data.edges:
    model.addConstr(gp.quicksum(x[x_i_p] for x_i_p in edges_paths[e]) <= data.vehicle_capacity*y[e])

  for office in solver_data.limited_offices:
    model.addConstr(gp.quicksum(x[x_i_p] for x_i_p in offices_paths[office])<=data.offices[office][1])

  # Решение
  model.setParam('OutputFlag', 0)
  start_time = time.time()
  model.optimize()
  sol_time = time.time() - start_time

  if model.status == GRB.OPTIMAL:
    flow = {product_path_id: x[product_path_id].x for product_path_id in solver_data.product_path_ids if x[product_path_id].x>0}
    v_count = {e: y[e].x for e in data.edges}
    obj_val = model.objVal
    return flow, v_count ,obj_val
  else:
    return None, None, None

In [14]:
def solve_min_cost_flow(data, solver_data): # Реализация солвера без учета ограничения на максимальный вес перегрузки
  # Создаем модель
  model = pulp.LpProblem('MinCostFlow', pulp.LpMinimize)

  # Переменные
  x = {}
  for pid in solver_data.product_path_ids:
        lb = solver_data.flow_lb[pid]
        ub = solver_data.flow_ub[pid]
        x[pid] = pulp.LpVariable(f"flow_{pid}", lowBound=lb, upBound=ub, cat='Continuous')

  y = pulp.LpVariable.dicts('vehicles_count', data.edges, lowBound=0, cat='Integer')

  # Целевая функция
  model += (
      pulp.lpSum(data.edges_price[e] * y[e] for e in data.edges) +
      pulp.lpSum(
          pulp.lpSum(data.offices[v][0] * x[x_i_p] for v in solver_data.paths[x_i_p][1:-1])
          for x_i_p in solver_data.product_path_ids
      ),
      "Total_Cost"
  )

  # Ограничения
  # 1. Поток для каждого продукта должен удовлетворять спрос
  for product in data.products_ids:
      model += (
          pulp.lpSum(x[(product, p_i)] for p_i in range(solver_data.product_path_count[product])) == data.products[product][2],
          f"Demand_{product}"
      )

  # 2. Подготовка edges_paths и offices_paths
  edges_paths = {edge: set() for edge in data.edges}
  offices_paths = {office: set() for office in data.offices_ids}
  for product_path_id, path in solver_data.paths.items():
      for i in range(len(path) - 1):
          edges_paths[(path[i], path[i + 1])].add(product_path_id)
          if i != 0:
              offices_paths[path[i]].add(product_path_id)

  # 3. Ограничение на вместимость транспорта
  for e in data.edges:
      model += (
          pulp.lpSum(x[x_i_p] for x_i_p in edges_paths[e]) <= data.vehicle_capacity * y[e],
          f"Capacity_{e}"
      )

  # 4. Ограничение на пропускную способность узлов (если есть limited_offices)
  for office in solver_data.limited_offices:
      model += (
          pulp.lpSum(x[x_i_p] for x_i_p in offices_paths[office]) <= data.offices[office][1],
          f"Office_Capacity_{office}"
      )


  # Используем HiGHS как солвер
  solver = pulp.HiGHS(
    msg=True,
    mip=True,              
    timeLimit=7200.0,
    mip_rel_gap = 0.01,
    parallel = 1)          
  
  print(f"Переменных: {len(model.variables())}")
  print(f"Ограничений: {len(model.constraints)}")

  model.solve(solver)

  # Проверка статуса решения
  if pulp.LpStatus[model.status] == 'Optimal':
      flow = {product_path_id: x[product_path_id].varValue for product_path_id in solver_data.product_path_ids if x[product_path_id].varValue > 0}
      v_count = {e: y[e].varValue for e in data.edges}
      obj_val = pulp.value(model.objective)
      return flow, v_count, obj_val
  else:
      return None, None, None

In [15]:
def solve_min_cost_flow_or_tools(data, solver_data): # Реализация солвера без учета ограничения на максимальный вес перегрузки

  # Создаем модель
  model = cp_model.CpModel()

  # Переменные
  x = {}
  for pid in solver_data.product_path_ids:
        lb = solver_data.flow_lb[pid]
        ub = solver_data.flow_ub[pid]
        lb_val = 0 if lb is None else lb
        ub_val = 10000 if ub is None else ub
        x[pid] = model.NewIntVar(int(lb_val), int(ub_val), f"flow_{pid}")

  y = {e: model.NewIntVar(0, 1000000, f"vehicles_{e}") for e in data.edges}

  # Целевая функция
  obj_terms = []
  for e in data.edges:
    obj_terms.append(int(data.edges_price[e]) * y[e])
  for pid in solver_data.product_path_ids:
        path = solver_data.paths[pid]
        coeff = sum(int(data.offices[v][0]) for v in path[1:-1])
        if coeff != 0:
            obj_terms.append(coeff * x[pid])
  model.Minimize(sum(obj_terms))

  # Ограничения
  # 1. Поток для каждого продукта должен удовлетворять спрос
  for product in data.products_ids:
        pids = [pid for pid in solver_data.product_path_ids if hasattr(pid, '__getitem__') and pid[0] == product] \
               if any(hasattr(pid, '__getitem__') for pid in solver_data.product_path_ids) else \
               [pid for pid in solver_data.product_path_ids if str(pid).startswith(str(product))]
        if not pids:
            count = solver_data.product_path_count[product]
            pids = [(product, i) for i in range(count) if (product, i) in x]
        model.Add(sum(x[pid] for pid in pids) == int(data.products[product][2]))

  # 2. Подготовка edges_paths и offices_paths
  edges_paths = {edge: set() for edge in data.edges}
  offices_paths = {office: set() for office in data.offices_ids}
  for product_path_id, path in solver_data.paths.items():
      for i in range(len(path) - 1):
          edges_paths[(path[i], path[i + 1])].add(product_path_id)
          if i != 0:
              offices_paths[path[i]].add(product_path_id)

  # 3. Ограничение на вместимость транспорта
  for e in data.edges:
        pset = edges_paths.get(e, set())
        if pset:
            model.Add(sum(x[pid] for pid in pset) <= int(data.vehicle_capacity) * y[e])


  # 4. Ограничение на пропускную способность узлов (если есть limited_offices)
  for office in getattr(solver_data, 'limited_offices', []):
        pset = offices_paths.get(office, set())
        cap = int(data.offices[office][1])
        if pset:
            model.Add(sum(x[pid] for pid in pset) <= cap)


  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  if status == cp_model.OPTIMAL:
        flow = {pid: solver.Value(x_var) for pid, x_var in x.items() if solver.Value(x_var) > 0}
        v_count = {e: solver.Value(y_var) for e, y_var in y.items()}
        obj_val = solver.ObjectiveValue()
        return flow, v_count, obj_val
  else:
      return None, None, None

## 2-3. Разгрузка перегруженых узлов и решение итогового солвера

На данном этапе происходит процесс разгрузки перегруженных узлов.

Первым делом находим перегруженные узлы. Находим узел с наибольшим числом перегруза. Сортируем все пути через него по убыванию количества товара, которого перевозят по ним. Вычитаем данные значения количества товаров в порядке убывания из веса узла пока узел не перестанет быть перегруженным. Сохраним веса не удаленных путей, чтобы больше не проводить вычислений по этому складу.

Удаляем из графа этот перегруженный узел. Для каждого товара находим k-кратчайших новых путей. Грубо говоря, на данном этапе мы строим новые обходные пути перегруженного узла.

Для путей из прошлого решения (которые проходят через перегруежнный узел) и новых найденных решаем задачу через солвер. Так как новые пути не проходят через перегруженный узел, то новое решение уже не перегружает этот узел. 

Далее находим новый перегруженный узел и проделываем теже действия, только кратчайшие пути находим уже в графе без прошлого перегруженного и этого узлов. Повторяем эти действия, пока все узлы не перестанут быть перегруженными.

На протяжении работы вышеописанного цикла разгрузки сохраняем все построенные пути. И в самом конце запускаем солвер со всеми накопленными путями и ограничениями на перегруз.

In [32]:
class OverloadData: # Данные, расчитываемые при перегрузке вершин
  def __init__(self):
    self.overload_paths_flow = []
    self.overload_office = -1

In [17]:
def calc_overloads(data, solver_data, flow, overload_offices): #Расчитать необходимые данные для процесса разгрузки
  overloads = OverloadData()

  office_weight = {office: 0 for office in data.offices_ids}
  office_paths = {office: set() for office in data.offices_ids}
  overloads_flow = flow.copy()

  for product_path_id, weight in flow.items():
    for office in solver_data.paths[product_path_id][1:-1]:
      office_weight[office] += weight
      office_paths[office].add(product_path_id)

  overload_office_weight = {office: office_weight[office] - data.offices[office][1] for office in data.offices_ids }
  max_office, max_overload_weight = max(overload_office_weight.items(), key=lambda x: x[1])

  if(max_overload_weight<=0):
    return overloads
  
  overloads.overload_office = max_office
  overload_offices.add(max_office)

  sorted_paths = sorted(office_paths[max_office], key=lambda x: flow[x], reverse=True)

  while max_overload_weight>0 and len(sorted_paths)>0:
    path_to_delete = sorted_paths[0]

    weight_to_remove = min(max_overload_weight, overloads_flow[path_to_delete])

    overloads_flow[path_to_delete]-=weight_to_remove
    max_overload_weight-=weight_to_remove
    if max_overload_weight > 0:
      overloads_flow.pop(path_to_delete)
      sorted_paths.pop(0)
  
  overloads_paths = set()
  for office in overload_offices:
    for path_id in office_paths[office]:
      if path_id in overloads_flow:
        overloads_paths.add(path_id)

  for path_id in overloads_paths:
    overloads.overload_paths_flow.append((path_id[0], solver_data.paths[path_id], overloads_flow[path_id]))

  return overloads


In [18]:
def find_edges_with_overload_offices(edges_with_transfer_price, data, overload_offices): # Найти ребра, которые необходимо исключить в связи с исключением вершины графа
  edges_without_overload = edges_with_transfer_price.copy()
  for edge in edges_with_transfer_price:
    if edge[0] in overload_offices or edge[1] in overload_offices:
      edges_without_overload.pop(edge)

  return edges_without_overload

In [19]:
def calc_paths_overload(edges_with_transfer_price, data, limited_offices, count_of_paths): # Расчет путей при разргрузке
  paths = {product : [] for product in data.products_ids}

  for product in data.products_ids:
    product_data = data.products[product]
    overload_offices = limited_offices.copy()
    overload_offices.discard(product_data[0])
    overload_offices.discard(product_data[1])
    edges_for_product = find_edges_with_overload_offices(edges_with_transfer_price, data, overload_offices)
    paths[product] = calc_product_paths(edges_for_product, product_data[0], product_data[1], count_of_paths, data.offices)

  return paths

In [20]:
def make_overload_solver_data(paths, products, overload_paths_flow): # Формирование данных для солвера
  solver_data = SolverData()

  for product, path, weight in overload_paths_flow:
    path_id = (product, len(paths[product]))
    solver_data.flow_lb[path_id] = weight
    solver_data.flow_ub[path_id] = weight
    paths[product].append(path)

  max_ub = -1
  for product, product_data in products.items():
    max_ub = max(max_ub, product_data[2])

  for product, product_paths in paths.items():
    solver_data.product_path_count[product] = len(product_paths)
    for i, path in enumerate(product_paths):
      solver_data.product_path_ids.append((product, i))
      solver_data.paths[(product, i)] = path
      if (product, i) not in solver_data.flow_lb:
        solver_data.flow_lb[(product, i)] = 0
        solver_data.flow_ub[(product, i)] = max_ub


  return solver_data

In [21]:
def get_string_from_path(path): # Вспомогательны метод для красивово вывода в файл
  string = '('
  for i in range(len(path)-1):
    string += str(int(path[i]))+', '
  string += str(int(path[-1]))+')'
  return string

In [22]:
def write_result_into_file(file, flow, v_count , obj_val, sol_time, solver_data): # Запись результата в файл
  with open(file, 'w') as f:
    f.write('product, path, volume\n')
    for product_path_id, weight in flow.items():
      f.write(str(product_path_id[0])+', '+get_string_from_path(solver_data.paths[product_path_id])+', '+str(weight)+'\n')
    f.write('\nedge, vehicle_count\n')
    for edge, count in v_count.items():
      if(count>0):
        f.write('('+str(int(edge[0]))+', '+str(int(edge[1]))+'), '+str(count)+'\n')
    f.write('\nResult: '+str(obj_val)+'\n')
    f.write('\nSolution time: '+str(sol_time)+'\n')

In [23]:
def save_to_csv(file, flow, data, solver_data):
    with open(file, 'w') as f:
        rows = [{'src': data.products[product][0], 'dst': data.products[product][1], 'volume': volume, 'path_nodes': list(map(int, solver_data.paths[(product, path_id)]))} for (product, path_id), volume in flow.items()]
        df = pd.DataFrame(rows)
        df.to_csv(file, index=False, encoding='utf-8')

Следующий метод и реализует весь процесс решения задачи

In [24]:
def solve_MCF(vehicle_capacity, folder, output_file, k1, k2):
  start_time = time.time()
  data = Data(vehicle_capacity,folder)
  edges_with_transfer_price = count_price_of_edjes(data.edges_price, data.offices)
  paths = calc_paths(edges_with_transfer_price, data, k1)
  solver_data = make_solver_data(paths, data.products)
  
  limited_offices = set()
  available_paths = {product : [] for product in data.products_ids}
  for product in data.products_ids:
      available_paths[product]+=paths[product]
  step=1

  while (True):
    print(len(solver_data.paths))
    flow, v_count, obj_val = solve_min_cost_flow(data, solver_data)

    if(flow == None):
      print ("Грустная история(")
      break
    print (step,". ", obj_val)

    step+=1
    overloads = calc_overloads(data, solver_data, flow, limited_offices)
    if (overloads.overload_office == -1):
      break
    
    paths = calc_paths_overload(edges_with_transfer_price, data, limited_offices, k2)

    for product in data.products_ids:
      available_paths[product]+=paths[product]

    solver_data = make_overload_solver_data(paths, data.products, overloads.overload_paths_flow)

  sol_time = time.time() - start_time
  write_result_into_file(output_file+'prefinal.txt', flow, v_count , obj_val, sol_time, solver_data)
  save_to_csv(output_file+'prefinal.csv', flow, data, solver_data)

  print ('The last step')  
  for product in data.products_ids:
      unique_paths = list(dict.fromkeys(map(tuple, available_paths[product])))
      available_paths[product] = [list(path) for path in unique_paths]
    
  solver_data = make_solver_data(available_paths, data.products)
  solver_data.limited_offices=data.offices_ids
  print(len(solver_data.paths))
  flow, v_count, obj_val = solve_min_cost_flow(data, solver_data)

  sol_time = time.time() - start_time
  if(flow != None):
    write_result_into_file(output_file+'.txt', flow, v_count , obj_val, sol_time, solver_data)
    save_to_csv(output_file+'.csv', flow, data, solver_data)
    print('Result: '+str(obj_val)+'\n')
    print('Solution time: '+str(sol_time)+'\n')


## Результаты

In [25]:
solve_MCF(60, 'MCF/4_nodes/', 'results/4_nodes_result', 3,2)

7
Переменных: 12
Ограничений: 10
1 .  24080.0
7
Переменных: 12
Ограничений: 10
2 .  24080.0
The last step
7
Переменных: 12
Ограничений: 14
Result: 24080.0

Solution time: 0.09093832969665527



In [26]:
solve_MCF(90, 'MCF/10_nodes/', 'results/10_nodes_result', 5, 5)

130
Переменных: 202
Ограничений: 98
1 .  1064985083.4299996
141
Переменных: 213
Ограничений: 98
2 .  1349961523.1499996
146
Переменных: 218
Ограничений: 98
3 .  1446189765.3375998
150
Переменных: 222
Ограничений: 98
4 .  1503590142.3910003
153
Переменных: 225
Ограничений: 98
5 .  1751862969.0209997
155
Переменных: 227
Ограничений: 98
6 .  1818927111.8209999
155
Переменных: 227
Ограничений: 98
7 .  1818927111.8209999
The last step
395
Переменных: 467
Ограничений: 107
Result: 1106157742.8409762

Solution time: 6.3259851932525635



In [31]:
solve_MCF(90, 'MCF/20_nodes/', 'results/20_nodes_result', 5, 5)

575
Переменных: 917
Ограничений: 457
1 .  11390665494.512
604
Переменных: 946
Ограничений: 457
2 .  12137552860.0026
615
Переменных: 957
Ограничений: 457
3 .  12266482786.006588
628
Переменных: 970
Ограничений: 457
4 .  12835601309.190203
638
Переменных: 980
Ограничений: 457
5 .  13199058400.564796
657
Переменных: 999
Ограничений: 457
6 .  14135842444.516798
666
Переменных: 1008
Ограничений: 457
7 .  14674412379.313002
674
Переменных: 1016
Ограничений: 457
8 .  15664397466.933998
682
Переменных: 1024
Ограничений: 457
9 .  19489527309.153183
690
Переменных: 1032
Ограничений: 457
10 .  20046118278.855988
695
Переменных: 1037
Ограничений: 457
11 .  22354685424.244
698
Переменных: 1040
Ограничений: 457
12 .  22977749271.555992
706
Переменных: 1048
Ограничений: 457
13 .  24088298387.785995
718
Переменных: 1060
Ограничений: 457
14 .  28398117353.225506
723
Переменных: 1065
Ограничений: 457
15 .  31068600199.825996
726
Переменных: 1068
Ограничений: 457
16 .  33796937046.145996
728
Переменных:

In [28]:
solve_MCF(90, 'MCF/50_nodes/', 'results/50_nodes_result1', 2, 2)

4212
Переменных: 13914
Ограничений: 11808
1 .  135283256234.99023
4478
Переменных: 14180
Ограничений: 11808
2 .  138393522913.19864
4612
Переменных: 14314
Ограничений: 11808
3 .  139094869867.146
4676
Переменных: 14378
Ограничений: 11808
4 .  149373910629.85922
4724
Переменных: 14426
Ограничений: 11808
5 .  153571405487.87665
4755
Переменных: 14457
Ограничений: 11808
6 .  158802491217.70834
4772
Переменных: 14474
Ограничений: 11808
7 .  160398898143.25323
4818
Переменных: 14520
Ограничений: 11808
8 .  163883039824.14645
4835
Переменных: 14537
Ограничений: 11808
9 .  166131958717.88715
4844
Переменных: 14546
Ограничений: 11808
10 .  169688914429.94717
4858
Переменных: 14560
Ограничений: 11808
11 .  172832626410.30865
4866
Переменных: 14568
Ограничений: 11808
12 .  178238308172.12277
4885
Переменных: 14587
Ограничений: 11808
13 .  181295974995.67154
4906
Переменных: 14608
Ограничений: 11808
14 .  182661842917.0236
4919
Переменных: 14621
Ограничений: 11808
15 .  185306581242.94855
4948
Пе

In [30]:
solve_MCF(90, 'MCF/140_nodes/', 'results/140_nodes_results', 1, 1)

3989
Переменных: 23449
Ограничений: 23449
1 .  225012744882.0385
4290
Переменных: 23750
Ограничений: 23449
2 .  245244819601.6467
4557
Переменных: 24017
Ограничений: 23449
3 .  249650798568.92896
4605
Переменных: 24065
Ограничений: 23449
4 .  256602852116.34204
4744
Переменных: 24204
Ограничений: 23449
5 .  257763555796.34073
4837
Переменных: 24297
Ограничений: 23449
6 .  274930004772.4183
4911
Переменных: 24371
Ограничений: 23449
7 .  279819624722.1227
4936
Переменных: 24396
Ограничений: 23449
8 .  284576391753.9555
4958
Переменных: 24418
Ограничений: 23449
9 .  286940613153.33997
4975
Переменных: 24435
Ограничений: 23449
10 .  290150356073.99133
4988
Переменных: 24448
Ограничений: 23449
11 .  292171523957.7279
5012
Переменных: 24472
Ограничений: 23449
12 .  299061837500.3859
5031
Переменных: 24491
Ограничений: 23449
13 .  303715481704.8728
5060
Переменных: 24520
Ограничений: 23449
14 .  306550973930.5106
5063
Переменных: 24523
Ограничений: 23449
15 .  313852989749.90344
5094
Переменн

## Точное решение

Здесь я добавила решение задачи точным образом с помощью потоковой постановки задачи

In [253]:
class OptimSolverData:
  def __init__(self):
    self.product_edge_ids = [] # массив (индекс товара - индекс ребра)
    self.edge_in = {}
    self.edge_out = {}

In [254]:
def solve_optim_MCF(vehicle_capacity, folder):
  start_time = time.time()
  data = Data(vehicle_capacity, folder)

  optim_solver_data = OptimSolverData()
  optim_solver_data.edge_in = {office: set() for office in data.offices_ids}
  optim_solver_data.edge_out = {office: set() for office in data.offices_ids}
  for edge in data.edges:
    for product in data.products_ids:
      optim_solver_data.product_edge_ids.append((edge, product))
    for office in data.offices_ids:
      optim_solver_data.edge_out[edge[0]].add(edge)
      optim_solver_data.edge_in[edge[1]].add(edge)

  flow, v_count , obj_val = ground_truth_solve_min_cost_flow(data, optim_solver_data)
  sol_time = time.time() - start_time
  print('Result: '+str(obj_val)+'\n')
  print('Solution time: '+str(sol_time)+'\n')


In [255]:
def ground_truth_solve_min_cost_flow(data, solver_data):

  model = gp.Model('MinCostFlow')

  # Переменные
  x = model.addVars(solver_data.product_edge_ids, name='flow', vtype = GRB.CONTINUOUS)
  y = model.addVars(data.edges, name='vehicles count', lb=0, vtype = GRB.INTEGER)

  # Целевая функция
  model.setObjective(
      gp.quicksum(data.edges_price[e]*y[e] for e in data.edges) + gp.quicksum(data.offices[office][0] * gp.quicksum(x[x_i_p] for x_i_p in solver_data.product_edge_ids if data.products[x_i_p[1]][0] != office and data.products[x_i_p[1]][1]!= office and x_i_p[0] in solver_data.edge_in[office]) for office in data.offices_ids),
        GRB.MINIMIZE
    )

  # Ограничения

  for product in data.products_ids:
    for office in data.offices_ids:
      if (office == data.products[product][0]):
        model.addConstr(gp.quicksum(x[(edge, product)] for edge in solver_data.edge_out[office]) - gp.quicksum(x[(edge, product)] for edge in solver_data.edge_in[office]) == data.products[product][2])
      elif (office == data.products[product][1]):
        model.addConstr(gp.quicksum(x[(edge, product)] for edge in solver_data.edge_out[office]) - gp.quicksum(x[(edge, product)] for edge in solver_data.edge_in[office]) == -data.products[product][2])
      else:
        model.addConstr(gp.quicksum(x[(edge, product)] for edge in solver_data.edge_out[office]) - gp.quicksum(x[(edge, product)] for edge in solver_data.edge_in[office]) == 0)


  for edge in data.edges:
    model.addConstr(gp.quicksum(x[(edge, product)] for product in data.products_ids) <= data.vehicle_capacity*y[edge])

  for office in data.offices_ids:
    model.addConstr(gp.quicksum(x[x_i_p] for x_i_p in solver_data.product_edge_ids if data.products[x_i_p[1]][0] != office and data.products[x_i_p[1]][1]!= office and x_i_p[0] in solver_data.edge_in[office])<= data.offices[office][1])

  # Решение
  model.setParam('OutputFlag', 0)
  model.optimize()

  if model.status == GRB.OPTIMAL:
    flow = {product_edge_id: x[product_edge_id].x for product_edge_id in solver_data.product_edge_ids if x[product_edge_id].x>0}
    v_count = {e: y[e].x for e in data.edges}
    obj_val = model.objVal
    return flow, v_count ,obj_val
  else:
    return None, None, None

In [257]:
solve_optim_MCF(90, 'MCF/10_nodes/')

NameError: name 'gp' is not defined

In [ ]:
solve_optim_MCF(90, 'MCF/20_nodes/')

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
solve_optim_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/50_nodes/')

In [ ]:
solve_optim_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/140_nodes/')